# ETL Project:

### Extraction:

our original data sources was formatted CSV.
we read the CSV files one by one to each different dataframes.
we finally displayed the information that has been extracted to the dataframes using pandas.

### Transformation:

The type of transformation we can perform with our data was cleaning, joining, filtering, aggregating etc.
we eliminated duplicate data and columns those are not containing appropriate data for our analysis.
We created transformed dataframes using fetching the data from actually extracted data.

### Loading:
before loading the data into the database we need to connect to the sql database.
we created an engine for the connection importing 'pymysql'.
Now exact data we loaded to our own database respective tables.


In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
stores_file = "../walmart/stores.csv"
stores_df = pd.read_csv(stores_file)

#stores_df.set_index('Store',inplace = True)
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [3]:
features_file = "../walmart/features.csv"
features_df = pd.read_csv(features_file)

#features_df.set_index('Date',inplace = True)
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
info_file = "../walmart/train.csv"
info_df = pd.read_csv(info_file)

#info_df.set_index('Date',inplace = True)
info_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


# Transform county DataFrame

In [5]:
# Create a filtered dataframe from specific columns
stores_cols = ["Store", "Size"]
stores_transformed= stores_df[stores_cols].copy()

# Rename the column headers
stores_transformed = stores_transformed.rename(columns={"Store": "store_id",
                                                          "Size": "size"})

# Clean the data by dropping duplicates and setting the index
stores_transformed.drop_duplicates("store_id", inplace=True)
stores_transformed.set_index("store_id", inplace=True)

stores_transformed.head()

,size
store_id,
1,151315
2,202307
3,37392
4,205863
5,34875


In [6]:
# Create a filtered dataframe from specific columns
features_cols = ["Store", "Date", "Temperature", "Fuel_Price", "CPI", "Unemployment"]
features_transformed= features_df[features_cols].copy()

# Rename the column headers
features_transformed = features_transformed.rename(columns={"Store": "store_id",
                                                          "Date": "date",
                                                           "Temperature":"temperature",
                                                           "Fuel_Price":"fuel_price",
                                                           "CPI":"CPI",
                                                           "Unemployment":"unemployment"})

# Clean the data by dropping duplicates and setting the index
features_transformed.drop_duplicates("date", inplace=True)
features_transformed.set_index("date", inplace=True)

features_transformed.head()

,store_id,temperature,fuel_price,CPI,unemployment
date,,,,,
2010-02-05,1,42.31,2.572,211.096358,8.106
2010-02-12,1,38.51,2.548,211.242170,8.106
2010-02-19,1,39.93,2.514,211.289143,8.106
2010-02-26,1,46.63,2.561,211.319643,8.106
2010-03-05,1,46.50,2.625,211.350143,8.106


In [7]:
# Create a filtered dataframe from specific columns
info_cols = ["Store", "Date", "Weekly_Sales", "IsHoliday"]
info_transformed= info_df[info_cols].copy()

# Rename the column headers
info_transformed = info_transformed.rename(columns={"Store": "store_id",
                                                          "Date": "date",
                                                           "Weekly_Sales":"weekly_sales",
                                                           "IsHoliday":"is_holiday"})

# Clean the data by dropping duplicates and setting the index
info_transformed.drop_duplicates("date", inplace=True)
info_transformed.set_index("date", inplace=True)

info_transformed.head()

,store_id,weekly_sales,is_holiday
date,,,
2010-02-05,1,24924.50,False
2010-02-12,1,46039.49,True
2010-02-19,1,41595.55,False
2010-02-26,1,19403.54,False
2010-03-05,1,21827.90,False


# Create database connection

In [16]:
import pymysql
connection_string = "root:payel@localhost/project2_db"
engine = create_engine(f'mysql://{connection_string}', pool_size=10, max_overflow=20)

In [19]:
# Confirm tables
engine.table_names()

['features_table', 'info_table', 'stores_table']

# Load DataFrames into database

In [20]:
stores_transformed.to_sql(name='stores_table', con=engine, if_exists='append', index=True)

In [21]:
features_transformed.to_sql(name='features_table', con=engine, if_exists='append', index=True)

In [22]:
info_transformed.to_sql(name='info_table', con=engine, if_exists='append', index=True)